In [1]:
import pandas as pd

C:\Users\user\AppData\Local\Temp\ipykernel_23996\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# This example demmonstrates how we transform text data into numeric quantities for analysis

# Data Source
linkBus = "https://github.com/DS-100/textbook/raw/master/content/datasets/SF_Restaurant_Inspections/businesses.csv"
linkInsp = "https://github.com/DS-100/textbook/raw/master/content/datasets/SF_Restaurant_Inspections/inspections.csv"
linkViol = "https://github.com/DS-100/textbook/raw/master/content/datasets/SF_Restaurant_Inspections/inspections.csv"

In [3]:
# QC : measurements and values
bus = pd.read_csv(linkBus, encoding='ISO-8859-1')
insp = pd.read_csv(linkInsp)
viol = pd.read_csv(linkViol)

print(bus.info())
print(insp.info())  #the date feature doesn't have proper value and data type
print(viol.info())  #the date feature doesn't have proper value and data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6406 entries, 0 to 6405
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   6406 non-null   int64  
 1   name          6406 non-null   object 
 2   address       6406 non-null   object 
 3   city          6406 non-null   object 
 4   state         6406 non-null   object 
 5   postal_code   6166 non-null   object 
 6   latitude      3270 non-null   float64
 7   longitude     3270 non-null   float64
 8   phone_number  5766 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 450.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14222 entries, 0 to 14221
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  14222 non-null  int64 
 1   score        14222 non-null  int64 
 2   date         14222 non-null  int64 
 3   type         14222 non-null  objec

In [4]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
1,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
2,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
3,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051
4,48,ART'S CAFE,747 IRVING ST,San Francisco,CA,94122,37.764013,-122.465749,+14156657440


In [5]:
len(bus.business_id.unique())

6406

In [6]:
#QC: Scope
bus.groupby(['postal_code']).size()
#there is a handfull of a features doesn't match the population
#since the problem is relatively minor, than leave the data as it  is (for while)

postal_code
00000          2
64110          1
92672          1
94013          2
94014          2
94066          1
94080          1
941            1
94101          2
94102        462
94103        552
941033148      1
94104        139
94105        232
94107        460
94108        228
94109        380
94110        596
941102019      1
94111        277
94112        195
94114        223
94115        243
94116         99
94117        204
94118        249
94120          1
94121        160
94122        273
94123        173
94124        191
94127         71
94129          2
94130          7
94131         49
94132        133
94133        426
94134         77
94143          5
94158         32
94188          4
94544          1
94545          1
94602          1
94621          1
95105          1
CA             2
Ca             1
dtype: int64

In [7]:
insp.head() #say we want to know whether inspectionsa happen more often on weekends or weekdays?

,business_id,score,date,type
0,19,94,20160513,routine
1,19,94,20171211,routine
2,24,98,20171101,routine
3,24,98,20161005,routine
4,24,96,20160311,routine


In [8]:
#piping
date_format = '%Y%m%d'

def parse_dates_and_years(df, column='date'):
    dates = pd.to_datetime(df[column], format=date_format)
    years = dates.dt.year
    return df.assign(timestamp=dates, year=years)

def extract_day_of_week(df, col='timestamp'):
    return df.assign(dow=df[col].dt.day_of_week)

In [9]:
insp = (pd.read_csv(linkInsp)
        .pipe(parse_dates_and_years)
        .pipe(extract_day_of_week))
insp.head()

,business_id,score,date,type,timestamp,year,dow
0,19,94,20160513,routine,2016-05-13,2016,4
1,19,94,20171211,routine,2017-12-11,2017,0
2,24,98,20171101,routine,2017-11-01,2017,2
3,24,98,20161005,routine,2016-10-05,2016,2
4,24,96,20160311,routine,2016-03-11,2016,4


In [15]:
viol = (pd.read_csv(linkViol)
        .pipe(parse_dates_and_years)) 
viol.head()

,business_id,score,date,type,timestamp,year
0,19,94,20160513,routine,2016-05-13,2016
1,19,94,20171211,routine,2017-12-11,2017
2,24,98,20171101,routine,2017-11-01,2017
3,24,98,20161005,routine,2016-10-05,2016
4,24,96,20160311,routine,2016-03-11,2016


In [19]:
#narrowing focus on year 2016
def subset_2016(df):
    return df.query("year==2016")

vio2016 = viol.pipe(subset_2016)
ins2016 = insp.pipe(subset_2016)

In [21]:
ins2016.head()
# what kind of failed QC happen in this data?
#how big problem cause by this failed QC?

,business_id,score,date,type,timestamp,year,dow
0,19,94,20160513,routine,2016-05-13,2016,4
3,24,98,20161005,routine,2016-10-05,2016,2
4,24,96,20160311,routine,2016-03-11,2016,4
6,45,78,20160104,routine,2016-01-04,2016,0
9,45,84,20160614,routine,2016-06-14,2016,1


In [22]:
len(ins2016.business_id.unique()) / len(ins2016.business_id)

0.77071467940474

mendekati 0 ada banyak redudansi, jika 1 tidak ada redudansi

In [24]:
len(ins2016.business_id)

5443

In [26]:
ins2016.groupby('business_id').size()

business_id
19       1
24       2
45       2
48       1
56       1
        ..
89931    1
90041    1
90096    1
90268    1
90269    1
Length: 4195, dtype: int64

In [25]:
len(ins2016.business_id)-len(ins2016.business_id.unique())

1248

In [28]:
vio2016.head(5)

,business_id,score,date,type,timestamp,year
0,19,94,20160513,routine,2016-05-13,2016
3,24,98,20161005,routine,2016-10-05,2016
4,24,96,20160311,routine,2016-03-11,2016
6,45,78,20160104,routine,2016-01-04,2016
9,45,84,20160614,routine,2016-06-14,2016


In [31]:
len(ins2016.business_id)

5443

In [33]:
ins2016.groupby('business_id').size().unique()

array([1, 2, 3], dtype=int64)

In [36]:
num_vios = (vio2016
            .groupby(['business_id', 'timestamp'])
            .size()
            .reset_index()
            .rename(columns={0: 'num_vio'}));
num_vios.head(3)

,business_id,timestamp,num_vio
0,19,2016-05-13,1
1,24,2016-03-11,1
2,24,2016-10-05,1


In [37]:
#left join the violation table on inspection table (left: as primary)
def left_join_vios(ins):
    return ins.merge(num_vios, on=['business_id', 'timestamp'], how='left')

ins_and_num_vios = ins2016.pipe(left_join_vios)
ins_and_num_vios

,business_id,score,date,type,timestamp,year,dow,num_vio
0,19,94,20160513,routine,2016-05-13,2016,4,1
1,24,98,20161005,routine,2016-10-05,2016,2,1
2,24,96,20160311,routine,2016-03-11,2016,4,1
3,45,78,20160104,routine,2016-01-04,2016,0,1
4,45,84,20160614,routine,2016-06-14,2016,1,1
...,...,...,...,...,...,...,...,...
5438,89931,100,20161207,routine,2016-12-07,2016,2,1
5439,90041,100,20161215,routine,2016-12-15,2016,3,1
5440,90096,91,20161229,routine,2016-12-29,2016,3,1
5441,90268,100,20161229,routine,2016-12-29,2016,3,1


In [38]:
# Count missing values in "score"
missing_scores = ins_and_num_vios["score"].isnull().sum()
print("Number of missing values in 'score':", missing_scores)

Number of missing values in 'score': 0


In [41]:
#Check berapa banyak missing value untuk score 100 pada table ins_and_num_vios
score_100 = ins_and_num_vios[ins_and_num_vios['score']==100]
missing_values_count = score_100['num_vio'].isna().sum()

print("Missing values untuk score 100 adalah",missing_values_count)

Missing values untuk score 100 adalah 0


In [43]:
print('number of missing value: ', ins_and_num_vios['num_vio'].isnull().sum())
print('number of records: ', len(ins_and_num_vios.business_id))
print('number of missing value(in percent): ', ins_and_num_vios['num_vio'].isnull().sum()/len(ins_and_num_vios.business_id)*100)

number of missing value:  0
number of records:  5443
number of missing value(in percent):  0.0


In [44]:
def zero_vios_for_perfect_scores(df):
    df = df.copy()
    df.loc[df['score'] == 100, 'num_vio'] = 0
    return df

ins_and_num_vios = (ins2016.pipe(left_join_vios)
                    .pipe(zero_vios_for_perfect_scores))

In [51]:
ins_and_num_vios['num_vio'].isnull().sum()

0

In [60]:
def make_vio_categories(vio):
    
    def has(term):
        return vio['description'].str.contains(term)

    return vio[['business_id', 'timestamp']].assign(
        high_risk        = has(r"high risk"),
        clean            = has(r"clean|sanit"),
        food_surface     = (has(r"surface") & has(r"\Wfood")),
        vermin           = has(r"vermin"),
        storage          = has(r"thaw|cool|therm|storage"),
        permit           = has(r"certif|permit"),
        non_food_surface = has(r"wall|ceiling|floor|surface"),
        human            = has(r"hand|glove|hair|nail"),
    )

vio_ctg = vio2016.pipe(make_vio_categories)
vio_ctg

KeyError: 'description'

In [56]:
vio_ctg = vio2016.pipe(make_vio_categories)
vio_ctg

KeyError: 'description'

In [45]:
#large number of missing value is corrected
#there is a handful mising value, what is means? how it affect our analysis?
ins_and_num_vios['num_vio'].isnull().sum()

0

dari data yang ada ini apakah score restoran lebih berdampak ke vermin than permmit

bagaimana cara membaca boxplot